In [1]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_classes
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from training_testing import validate_model, train_regular
from model_saver import model_saver, model_saver_wandb
import training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [4]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [5]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [6]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
hidden_size = 64
raceDB = build_dataset('gru_inputs_new2_pir_simple_SP.npy', hidden_size ,state_filter=["NSW"], margin_type='boosted_sftmin')
raceDB.create_new_weights_v2()
#raceDB.adjust_weights({"Dapto":10, "Gunnedah":10})

In [ ]:
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
raceDB.create_test_split_date(date)

Train examples 13565, Test examples 3071


In [ ]:
start_date = datetime.datetime.strptime("2019-12-01", "%Y-%m-%d").date()
end_date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()-datetime.timedelta(1)
period = start_date
batches = []
while start_date<end_date:
   
    print(f"{start_date=}")
    period = min(start_date+datetime.timedelta(90), end_date)
    print(f"{period=}")
    batches.append((start_date,period))
    start_date=period
    #batches.append((start_date,end_date))
# batches.append((period,end_date))
print(batches)

start_date=datetime.date(2019, 12, 1)
period=datetime.date(2020, 2, 29)
start_date=datetime.date(2020, 2, 29)
period=datetime.date(2020, 5, 29)
start_date=datetime.date(2020, 5, 29)
period=datetime.date(2020, 8, 27)
start_date=datetime.date(2020, 8, 27)
period=datetime.date(2020, 11, 25)
start_date=datetime.date(2020, 11, 25)
period=datetime.date(2021, 2, 23)
start_date=datetime.date(2021, 2, 23)
period=datetime.date(2021, 5, 24)
start_date=datetime.date(2021, 5, 24)
period=datetime.date(2021, 8, 22)
start_date=datetime.date(2021, 8, 22)
period=datetime.date(2021, 11, 20)
start_date=datetime.date(2021, 11, 20)
period=datetime.date(2022, 2, 18)
start_date=datetime.date(2022, 2, 18)
period=datetime.date(2022, 5, 19)
start_date=datetime.date(2022, 5, 19)
period=datetime.date(2022, 7, 31)
[(datetime.date(2019, 12, 1), datetime.date(2020, 2, 29)), (datetime.date(2020, 2, 29), datetime.date(2020, 5, 29)), (datetime.date(2020, 5, 29), datetime.date(2020, 8, 27)), (datetime.date(2020, 8, 27), 

In [ ]:
batch_races_ids = [] # list of race_ids
j = 0
current_batch = []
for i,r in enumerate(raceDB.raceIDs):   
    _,end_date = batches[j]
    race_date = raceDB.racesDict[r].race_date
    if race_date>end_date:
        print(end_date)

        batch_races_ids.append(current_batch)
        current_batch = []
        j += 1
        if j>len(batches)-1:
            break
    else:
        current_batch.append(r)
print(f"Train examples {[len(x) for x in batch_races_ids]}")

2020-02-29
2020-05-29
2020-08-27
2020-11-25
2021-02-23
2021-05-24
2021-08-22
2021-11-20
2022-02-18
2022-05-19
2022-07-31
Train examples [1396, 660, 1451, 1494, 1343, 1360, 1242, 1141, 1196, 1237, 974]


In [ ]:
train_dogs = []
train_dog_input = []
for bi, batch in enumerate(tqdm(batch_races_ids)):
    print(bi)
    print(batch)
    batch_dogs = []
    batch_dog_input = []
    for i in raceDB.dog_ids:
        dog = raceDB.dogsDict[i]
        train = [dog.races[x] for x in dog.races.keys() if x in batch]
        if train:
            batch_dogs.append(dog)
            batch_dog_input.append(train)
    train_dogs.append(batch_dogs)
    train_dog_input.append(batch_dog_input)

  0%|          | 0/11 [00:00<?, ?it/s]

0
['500539129', '500539130', '500539131', '500539132', '500539133', '500539134', '500539135', '500539136', '500539137', '500539138', '500539139', '500539140', '500539141', '500539142', '500539143', '500539144', '500541168', '500541169', '500541170', '500541171', '500541172', '500541173', '500541174', '500541175', '500541176', '500541177', '500541178', '500541179', '500541180', '500541181', '500541182', '500543140', '500543141', '500543142', '500543143', '500543144', '500543145', '500543146', '500543147', '500543148', '500543149', '500543150', '500543151', '500543152', '501809115', '501809116', '501809117', '501809118', '501809119', '501809120', '501809121', '501809122', '501809123', '501809124', '501809125', '501809126', '501809127', '501809128', '501809129', '502271152', '502271153', '502271154', '502271155', '502271156', '502271157', '502271158', '502271159', '502271160', '502271161', '502271162', '502271163', '502271164', '502271165', '502271166', '502271167', '502271168', '50256366

In [11]:
batch_races = [[raceDB.racesDict[r] for r in inner] for inner in batch_races_ids]

In [12]:
print(f"Train examples {[len(x) for x in train_dogs]}")
print(f"Train examples {[len(x) for x in train_dog_input]}")
print(f"Train examples {[len(x) for x in batch_races]}")
print(f"Train examples {[len(x) for x in batch_races_ids]}")

Train examples [1395, 1251, 1401, 1352, 1312, 1274, 1269, 1243, 1198, 1159, 1093]
Train examples [1395, 1251, 1401, 1352, 1312, 1274, 1269, 1243, 1198, 1159, 1093]
Train examples [1396, 660, 1451, 1494, 1343, 1360, 1242, 1141, 1196, 1237, 974]
Train examples [1396, 660, 1451, 1494, 1343, 1360, 1242, 1141, 1196, 1237, 974]


In [13]:
raceDB.create_dogs_test_split_date()

  0%|          | 0/3546 [00:00<?, ?it/s]

In [14]:
raceDB.attach_races_to_dog_input()

  0%|          | 0/3546 [00:00<?, ?it/s]

In [15]:
train_dog_input_stacked = [[torch.stack(n,0) for n in [[z.full_input.to('cpu') for z in inner] for inner in x]] for x in train_dog_input]

In [16]:
[sum([len(x) for x in n])/len([len(x) for x in n]) for n in train_dog_input_stacked]

[7.720430107526882,
 4.091926458832933,
 8.036402569593148,
 8.479289940828401,
 7.496951219512195,
 8.198587127158556,
 7.456264775413712,
 6.918744971842317,
 7.545909849749583,
 8.036238136324418,
 6.617566331198536]

In [17]:
raceDB.batches = {'num_batches':len(train_dogs),'dogs':train_dogs,'train_dog_input':train_dog_input,'batch_races':batch_races,'batch_races_ids':batch_races_ids}

In [23]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      wandb.define_metric("multibet profit", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy

      model = rnn_classes.GRUNetv2(205,config['hidden_size'], num_layers=config['num_layers'])

      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        model_name = prev_model_file
        model_loc = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_450.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        model.load_state_dict(model_data['model_state_dict'])
        config['parent model'] = prev_model_file

      raceDB.to_cuda()

      # criterion = nn.SmoothL1Loss(reduction='mean', beta=5)
      criterion = nn.CrossEntropyLoss(reduction='none')
      # optimizer = optim.Adam(model.parameters(), lr=0.0001)
      # optimizer = optim.Adadelta(model.parameters())
      optimizer = optim.RMSprop(model.parameters(), lr=config['learning_rate'])

      model = model.to(device)

      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      try:
        # train_double_loss_regular(model, dataset, criterion, optimizer, scheduler, config, crit2=custom_l2)
        training_testing_gru.train_regular_batch(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt) as error:
        print(error)
        print("finished Early")
        
      dataset.create_hidden_states_dict()
      raceDB.create_hidden_states_dict_v2()
      model_saver_wandb(model, optimizer, 450, 0.1, dataset.hidden_states_dict_gru_v6 , model_name="long nsw new  22000 RUN")
      if sweep:
        raceDB.reset_all_lstm_states
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [19]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':hidden_size,'num_layers':3,'batch_size': 750, 'dropout': 0.3, 'epochs': 10_000, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta':0.1,   'optimizer': 'adamW', 'validation_split': 0.1,'batch_before_backwards':7}

In [20]:
#(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static)
importlib.reload(training_testing_gru)
importlib.reload(rnn_classes)
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

<module 'rnn_classes' from 'c:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\Python\\pytorch\\New Model\\LSTM\\rnn_classes.py'>

In [21]:
raceDB.reset_hidden()

In [24]:
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

{'hidden_size': 64, 'num_layers': 3, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta': 0.1, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 7}
10000
{'hidden_size': 64, 'num_layers': 3, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta': 0.1, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 7}
GRUNetv2(
  (gru): GRU(205, 64, num_layers=3)
  (batchnorm): BatchNorm1d(205, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU()
  (fc0): Linear(in_features=64, out_features=1, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=512, out_features=64, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=64, out_features=8, bias=True)
  (output_fn): 

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

...

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]


finished Early
created path



ROI < 30,███▇▇▆▄▂▁▁▂▂▃▃▄▄▅▅▅▅▆▆▅▅▄▄▅▄▄▄▄▅▄▅▅▅▅▅▅▅
accuracy,▁▁▁▁▃▄▅▅▅▆▆▆▆▇▆▇▇▇▇▇▇██▇████████████████
loss_1,███▇▇▇▆▆▆▅▅▅▆▄▅▅▄▅▄▄▄▅▄▄▃▃▂▄▄▃▄▃▃▃▅▁▃▃▂▁
loss_val,███▇▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
multibet outlay < 30,▁▁▁▁▂▂▂▃▃▃▃▄▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇████
multibet profit,██▇▇▇▅▃▂▁▂▂▃▃▄▅▅▆▅▆▅▆▆▆▅▅▄▅▄▄▄▃▄▃▃▃▃▃▃▃▃
multibet profit < 30,███▇▇▆▄▂▁▁▂▂▂▃▄▄▄▄▄▅▅▅▅▄▄▃▄▃▃▃▂▃▃▃▃▃▄▄▃▃
multibet profit < 30 sd,▁▁▁▁▁▂▂▃▃▃▃▄▄▄▄▄▄▄▄▄▅▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇███
ROI < 30,-0.0609
accuracy,0.29339
loss_1,1.47858


In [60]:
torch.rand([1395, 3, 64])

tensor([[[0.5179, 0.2236, 0.2084, 0.2784, 0.8993],
         [0.8444, 0.0626, 0.8215, 0.2098, 0.9365],
         [0.6379, 0.4137, 0.5337, 0.8011, 0.7283],
         [0.1999, 0.4694, 0.6658, 0.9972, 0.8104]],

        [[0.5405, 0.2567, 0.8223, 0.4344, 0.5713],
         [0.5393, 0.1814, 0.0334, 0.2168, 0.5938],
         [0.0909, 0.7370, 0.2014, 0.0187, 0.1430],
         [0.3972, 0.1687, 0.1375, 0.2822, 0.6694]]])

In [54]:
x = torch.rand([1395, 3, 64])
x[100]

tensor([[0.8063, 0.3263, 0.7562, 0.6170, 0.7359, 0.7247, 0.5439, 0.1375, 0.2478,
         0.1298, 0.1895, 0.6188, 0.7420, 0.9008, 0.1561, 0.8211, 0.4709, 0.9060,
         0.2838, 0.7162, 0.5264, 0.6671, 0.4468, 0.6569, 0.6848, 0.3746, 0.8414,
         0.8390, 0.8334, 0.3610, 0.7512, 0.1326, 0.3059, 0.0528, 0.7480, 0.7685,
         0.2068, 0.4202, 0.2671, 0.9943, 0.1555, 0.7093, 0.0813, 0.6830, 0.9158,
         0.3681, 0.8037, 0.6071, 0.3701, 0.0017, 0.2201, 0.0897, 0.0543, 0.0500,
         0.9881, 0.5474, 0.0765, 0.7898, 0.2657, 0.6868, 0.1202, 0.7253, 0.2968,
         0.0872],
        [0.4535, 0.4473, 0.7741, 0.3486, 0.9705, 0.3233, 0.9132, 0.0945, 0.4637,
         0.4017, 0.3071, 0.9169, 0.3772, 0.7355, 0.4952, 0.2160, 0.3648, 0.7348,
         0.2724, 0.3336, 0.5534, 0.3980, 0.0452, 0.4098, 0.3261, 0.8853, 0.6989,
         0.2683, 0.6162, 0.4117, 0.7940, 0.5879, 0.6622, 0.8176, 0.8999, 0.4844,
         0.2186, 0.4508, 0.6352, 0.9218, 0.0813, 0.8006, 0.7236, 0.7562, 0.7872,
         0

In [62]:
x.transpose(0,1)

tensor([[[0.8498, 0.4020, 0.9583, 0.4514, 0.4580],
         [0.5111, 0.5342, 0.5242, 0.3669, 0.8515],
         [0.0810, 0.6693, 0.5742, 0.9641, 0.6108],
         [0.6941, 0.8263, 0.4805, 0.4337, 0.8516],
         [0.9443, 0.5173, 0.7642, 0.3691, 0.6490],
         [0.2108, 0.4809, 0.8014, 0.2201, 0.6892],
         [0.0286, 0.6668, 0.5270, 0.8786, 0.0809],
         [0.0610, 0.7338, 0.2901, 0.8206, 0.6751],
         [0.5132, 0.8296, 0.4745, 0.7060, 0.1847],
         [0.4632, 0.5434, 0.5882, 0.1138, 0.8577]],

        [[0.4781, 0.3791, 0.7527, 0.4415, 0.8549],
         [0.6714, 0.9857, 0.4572, 0.6984, 0.8337],
         [0.9497, 0.2893, 0.5985, 0.2363, 0.5117],
         [0.2088, 0.7307, 0.9596, 0.9961, 0.9558],
         [0.9646, 0.8617, 0.8939, 0.8692, 0.0475],
         [0.7707, 0.2142, 0.3504, 0.9086, 0.9170],
         [0.2874, 0.5333, 0.9116, 0.5944, 0.6149],
         [0.4638, 0.7077, 0.6762, 0.8274, 0.8461],
         [0.8679, 0.3246, 0.7424, 0.0362, 0.5181],
         [0.2847, 0.3835, 0.8

In [40]:
torch.zeros(3,5,10)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [41]:
len_test = len(raceDB.test_dog_ids)
test_idx = range(0,len_test)

# dog_inputs = [[z.full_input for z in inner] for inner in [x for x in raceDB.train_dogs.values()]]
# Uses train dogs here because only dogs in Train set will have new hidden values?? Does that make sense?
# no will jumble around but wont throw error in feeding model, because when building dogs sorted, 
# lengths will be made to match size of X(test)
# Shouldn't have much of an effect due to size of test set??
dogs = [x for x in  raceDB.test_dogs.values()]  #[Dog]
dog_input = [inner for inner in [x for x in raceDB.get_dog_test(test_idx)]] #[[DogInput]]

# for dog in dogs:
#     dog.hidden = dog.hidden.to('cuda:0')

train = [torch.stack(n,0) for n in [[z.full_input.to('cpu') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]]
X = pack_sequence(train, enforce_sorted=False).to('cuda:0')

dogs_sorted = [dogs[x] for x in X[3]]

In [42]:
hidden_in = torch.stack([x.hidden for x in dogs])

In [44]:
hidden_in.shape

torch.Size([1439, 3, 64])

In [46]:
hidden_in.reshape((3, 1439, 64))

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')

In [59]:
dogs = [x for x in  raceDB.test_dogs.values()][:5]

In [61]:
[d.hidden for d in dogs]

[tensor([ 1.0000, -1.0000, -1.0000,  1.0000,  1.0000, -1.0000,  1.0000,  0.9979,
         -1.0000, -1.0000,  0.9952, -1.0000, -1.0000,  1.0000, -1.0000,  1.0000,
         -1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000,  1.0000, -1.0000,
         -1.0000,  0.8690,  0.9982, -0.7986, -1.0000,  0.8913,  1.0000, -1.0000,
          1.0000,  1.0000, -1.0000,  1.0000,  0.9824,  0.9698,  1.0000, -1.0000,
         -1.0000,  1.0000,  1.0000, -1.0000,  1.0000,  1.0000, -1.0000,  1.0000,
          1.0000,  1.0000,  1.0000,  1.0000,  1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000,  1.0000, -1.0000, -1.0000,  1.0000, -0.0602, -1.0000],
        device='cuda:0', grad_fn=<SelectBackward0>),
 tensor([ 1.0000, -1.0000, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000,  1.0000,
         -1.0000,  1.0000, -0.9940, -1.0000, -1.0000,  1.0000, -1.0000,  1.0000,
         -1.0000, -1.0000, -1.0000,  1.0000, -1.0000, -1.0000,  1.0000,  0.9459,
          0.9924,  1.0000,  0.1046, -0.2391, -1.0000, -

In [64]:
set(dog.races.keys()).intersection(set(raceDB.test_race_ids))
set(dog.races.keys()).intersection(set(raceDB.test_race_ids))

{'815075293', '824844960', '824845220', '827795806'}

In [12]:
[r.race.raceid for r in dog.test]

['815014466',
 '819276815',
 '819276866',
 '820706206',
 '820706317',
 '822917139',
 '874633915',
 '877730550',
 '877730921']

In [58]:
for i in set(dog.races.keys()).intersection(set(raceDB.test_race_ids)):
    print(i)

827795806
824844960
824845220
815075293


In [53]:
[dog.races[x] for x in set(dog.races.keys()).intersection(set(raceDB.test_race_ids))]

In [54]:
dog.test

In [13]:
dog_input = [inner for inner in [x for x in raceDB.get_dog_train([0,1,3,4,5,6])]][2]

In [14]:
races = [r.race for r in dog_input]

In [15]:
pd.Series([r.race_date for r in races])

0     2019-12-01
1     2019-12-05
2     2019-12-08
3     2019-12-15
4     2019-12-19
5     2019-12-22
6     2020-01-05
7     2020-01-12
8     2020-01-19
9     2020-01-26
10    2020-02-02
11    2020-02-06
12    2020-02-13
13    2020-02-16
14    2020-02-24
15    2020-03-01
16    2020-03-05
17    2020-03-15
18    2020-03-22
19    2020-05-17
20    2020-05-24
21    2020-06-04
22    2020-06-14
23    2020-06-21
24    2020-06-28
25    2020-07-05
26    2020-07-23
27    2020-08-06
28    2020-08-09
29    2020-08-27
30    2020-08-31
31    2020-10-04
32    2020-10-08
33    2020-10-11
34    2020-10-15
35    2020-10-18
36    2020-11-01
37    2020-11-12
38    2020-11-15
39    2020-11-29
40    2020-12-02
41    2020-12-09
42    2020-12-20
43    2020-12-27
44    2020-12-30
45    2021-01-10
46    2021-01-17
47    2021-01-24
48    2021-01-31
49    2021-02-04
50    2021-02-07
51    2021-02-11
52    2021-02-14
53    2021-02-17
54    2021-02-25
55    2021-03-03
56    2021-03-09
57    2021-03-14
dtype: object

In [20]:
raceDB.create_hidden_states_dict_v2()

In [22]:
model_saver(model,optimizer,450,0.2,raceDB.hidden_states_dict_gru_v6, model_name="long run to use")

created path


In [21]:
raceDB.hidden_states_dict_gru_v6

{'300896606': tensor([-1.0000,  1.0000,  1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000,
         -1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          1.0000,  1.0000, -1.0000,  1.0000, -0.9875, -1.0000, -1.0000,  1.0000,
         -1.0000, -1.0000,  1.0000,  1.0000,  1.0000, -1.0000,  1.0000,  1.0000,
          1.0000, -1.0000,  1.0000,  1.0000, -1.0000, -1.0000,  1.0000,  1.0000,
          1.0000, -1.0000, -1.0000,  0.0000,  1.0000,  1.0000,  1.0000,  1.0000,
          1.0000,  1.0000,  1.0000, -1.0000,  1.0000,  1.0000,  0.9968,  0.0000,
          1.0000,  1.0000, -1.0000, -1.0000, -1.0000,  1.0000,  0.0000,  1.0000],
        device='cuda:0'),
 '326972142': tensor([-1.0000,  1.0000,  1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000,
         -1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          1.0000,  1.0000, -1.0000,  1.0000,  1.0000, -1.0000, -1.0000,  1.0000,
         -1.0000, -1.0000,  0.9379,  1.0000,  1.0000,  1

In [17]:
criterion = nn.CrossEntropyLoss(reduction='none')

In [23]:
training_testing_gru.validate_model_v2(model,raceDB,criterion)

Error: You must call wandb.init() before wandb.log()

In [15]:
torch.cuda.empty_cache()

: 

In [68]:
len_test = len(raceDB.test_dog_ids)
test_idx = range(0,len_test)

# dog_inputs = [[z.full_input for z in inner] for inner in [x for x in raceDB.train_dogs.values()]]
dogs = [x for x in  raceDB.test_dogs.values()]  #[Dog]
dog_input = [inner for inner in [x for x in raceDB.get_dog_test(test_idx)]] #[[DogInput]]

for dog in dogs:
    dog.hidden = dog.hidden.to('cuda:0')

train = [torch.stack(n,0) for n in [[z.full_input for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]]
X = pack_sequence(train, enforce_sorted=False)

dogs_sorted = [dogs[x] for x in X[3]]
hidden_in = torch.unsqueeze(torch.stack([x.hidden for x in dogs_sorted]).to('cuda:0'),0)

In [70]:
batchnorm = torch.nn.BatchNorm1d(196)

In [71]:
batchnorm(X)

AttributeError: 'PackedSequence' object has no attribute 'dim'

In [27]:
loss = nn.CrossEntropyLoss(reduction='none')
X = torch.rand(5,8)
Y = torch.rand(5,8)
loss(X,Y)

tensor([9.3107, 8.9455, 7.7104, 9.2013, 8.2859])

In [ ]:
for i,dog in enumerate(dog_input):
    dog_outputs = output[i]
    for j,di in enumerate(dog):
        di.output = dog_outputs[j]

In [ ]:
raceDB.margin_from_dog_to_race()

In [ ]:
torch.tensor(100.1).to('cuda:0')

tensor(100.1000, device='cuda:0')

In [ ]:
torch.tensor([100.1]).to('cuda:0')

tensor([100.1000], device='cuda:0')

In [ ]:
for r in raceDB.test_race_ids:
    race = raceDB.racesDict[r]
    outputs = []
    for d in race.dogs:
        print(d)
        print(d.output)
        outputs.append(d.output)
    break
    print(output)
    race.margins = torch.stack(outputs)

tensor([100.1000], device='cuda:0')
tensor([2.4370], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([2.4468], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([0.9137], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([1.4955], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([2.2570], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([2.8256], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([2.1537], device='cuda:0', grad_fn=<SelectBackward0>)


In [ ]:
torch.cat(outputs)

tensor([100.1000,   1.2703,   1.0713,   0.9075,   1.2456,   0.7438,   0.8092,
          1.6421], device='cuda:0', grad_fn=<CatBackward0>)

TypeError: max(): argument 'input' (position 1) must be Tensor, not list

In [ ]:
output[0]

tensor([[1.0182],
        [1.1218],
        [0.7392],
        [0.6944],
        [0.6863],
        [0.7809],
        [0.6623],
        [0.2770],
        [0.4004],
        [0.3841],
        [0.4378],
        [0.6323],
        [0.6126],
        [1.2783],
        [0.4362],
        [1.2451],
        [0.7298],
        [1.1868],
        [1.6842],
        [1.3961],
        [0.9783],
        [0.3406],
        [0.6231]], device='cuda:0', grad_fn=<AddmmBackward0>)

RuntimeError: stack expects each tensor to be equal size, but got [] at entry 0 and [1] at entry 1

In [ ]:
test_races = [raceDB.racesDict[x] for x in raceDB.test_race_ids][0]

In [ ]:
test

NameError: name 'test' is not defined

In [ ]:
print(test_races.margins)

tensor([100.1000,   2.4370,   2.4468,   0.9137,   1.4955,   2.2570,   2.8256,
          2.1537], device='cuda:0', grad_fn=<CatBackward0>)


In [ ]:
[x.dog.dog_name for x in test_races.dogs]

['no_name',
 'MR. MUGGINS',
 'RETREAD PETE',
 "HOW'S THE LUCK",
 'CHLOE WHO',
 'HOMEBUSH THAW',
 'HOMEBUSH LEROY',
 'GOLDSTAR HOPE']

In [ ]:
sweep_config = {"method": "random"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "hidden_size": {"value":64}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [100]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "CEL"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00001,
            "max": 0.001,
        },
        "l1_beta": {"value": 0.1
        },
        "batch_size": {
            'values': [50,125,250,500]
        },
        "batch_before_backwards": {
            'values': [5,10,15,50]
        }
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'ROI < 30'},
 'parameters': {'batch_before_backwards': {'values': [5, 10, 15, 50]},
                'batch_size': {'values': [50, 125, 250, 500]},
                'dropout': {'values': [0.3]},
                'epochs': {'values': [100]},
                'f1_layer_size': {'values': [256]},
                'f2_layer_size': {'values': [64]},
                'hidden_size': {'value': 64},
                'l1_beta': {'value': 0.1},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.001,
                                  'min': 1e-05},
                'len_data': {'value': 88865},
                'loss': {'values': ['CEL']},
                'num_layers': {'values': [2]},
                'optimizer': {'value': 'adamW'},
                'validation_split': {'value': 0.1}}}


{'method': 'random',
 'metric': {'name': 'ROI < 30', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'f1_layer_size': {'values': [256]},
  'f2_layer_size': {'values': [64]},
  'dropout': {'values': [0.3]},
  'len_data': {'value': 88865},
  'hidden_size': {'value': 64},
  'epochs': {'values': [100]},
  'validation_split': {'value': 0.1},
  'loss': {'values': ['CEL']},
  'num_layers': {'values': [2]},
  'learning_rate': {'distribution': 'uniform', 'min': 1e-05, 'max': 0.001},
  'l1_beta': {'value': 0.1},
  'batch_size': {'values': [50, 125, 250, 500]},
  'batch_before_backwards': {'values': [5, 10, 15, 50]}}}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="LSTM_sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

Create sweep with ID: udnlmppo
Sweep URL: https://wandb.ai/nickojelly/LSTM_sweeps/sweeps/udnlmppo


wandb: Agent Starting Run: ctlxz40f with config:
wandb: 	batch_before_backwards: 10
wandb: 	batch_size: 125
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 64
wandb: 	l1_beta: 0.1
wandb: 	learning_rate: 0.0004436610065404108
wandb: 	len_data: 88865
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_before_backwards': 10, 'batch_size': 125, 'dropout': 0.3, 'epochs': 100, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.0004436610065404108, 'len_data': 88865, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
100
{'batch_before_backwards': 10, 'batch_size': 125, 'dropout': 0.3, 'epochs': 100, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.0004436610065404108, 'len_data': 88865, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
smalll_lin_GRUNet(
  (batchnorm): BatchNorm1d(510, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc0): Linear(in_features=510, out_features=510, bias=True)
  (gru1): GRUCell(510, 64)
  (gru2): GRUCell(510, 64)
  (gru3): GRUCell(510, 64)
  (gru4): GRUCell(510, 64)
  (gru5): GRUCell(510, 64)
  (gru6): GRUCell(510, 64)
  (gru7): GRUCell(510, 64)
  (gru8): GRUCell(510, 64)
  (rl1): 

  0%|          | 0/100 [00:00<?, ?it/s]


Run ctlxz40f errored: RuntimeError('hidden0 has inconsistent hidden_size: got 128, expected 64')
wandb: ERROR Run ctlxz40f errored: RuntimeError('hidden0 has inconsistent hidden_size: got 128, expected 64')
wandb: Agent Starting Run: 3ky1vnz4 with config:
wandb: 	batch_before_backwards: 15
wandb: 	batch_size: 125
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 64
wandb: 	l1_beta: 0.1
wandb: 	learning_rate: 3.554693043910414e-05
wandb: 	len_data: 88865
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

  0%|          | 0/100 [00:00<?, ?it/s]


Run 3ky1vnz4 errored: RuntimeError('hidden0 has inconsistent hidden_size: got 128, expected 64')
wandb: ERROR Run 3ky1vnz4 errored: RuntimeError('hidden0 has inconsistent hidden_size: got 128, expected 64')
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: hf60k4q6 with config:
wandb: 	batch_before_backwards: 50
wandb: 	batch_size: 500
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 64
wandb: 	l1_beta: 0.1
wandb: 	learning_rate: 0.0004939104475117235
wandb: 	len_data: 88865
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_before_backwards': 50, 'batch_size': 500, 'dropout': 0.3, 'epochs': 100, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.0004939104475117235, 'len_data': 88865, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
100
{'batch_before_backwards': 50, 'batch_size': 500, 'dropout': 0.3, 'epochs': 100, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.0004939104475117235, 'len_data': 88865, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
smalll_lin_GRUNet(
  (batchnorm): BatchNorm1d(510, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc0): Linear(in_features=510, out_features=510, bias=True)
  (gru1): GRUCell(510, 64)
  (gru2): GRUCell(510, 64)
  (gru3): GRUCell(510, 64)
  (gru4): GRUCell(510, 64)
  (gru5): GRUCell(510, 64)
  (gru6): GRUCell(510, 64)
  (gru7): GRUCell(510, 64)
  (gru8): GRUCell(510, 64)
  (rl1): 

  0%|          | 0/100 [00:00<?, ?it/s]


Run hf60k4q6 errored: RuntimeError('hidden0 has inconsistent hidden_size: got 128, expected 64')
wandb: ERROR Run hf60k4q6 errored: RuntimeError('hidden0 has inconsistent hidden_size: got 128, expected 64')
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


In [ ]:

stop
model = GRUNet(203,64).to(device)
model.load_state_dict(model_state_dict)

NameError: name 'stop' is not defined

In [ ]:

dataset.create_hidden_states_dict()